In [42]:
!pip install pandas requests

In [43]:
pip install ntscraper

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [45]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
import json
import firebase_admin
from firebase_admin import credentials, firestore, db
from bs4 import BeautifulSoup
from ntscraper import Nitter

In [46]:
service_account_path = os.getenv('FIREBASE_SERVICE_ACCOUNT_KEY')

In [47]:
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

In [48]:
db = firestore.client()

Testing instances: 100%|██████████| 1/1 [00:04<00:00,  4.54s/it]


In [61]:
def getTweets(name, modes, num):
    scraper = Nitter()
    scraper.working_instances = ['https://nitter.net']
    total = scraper.get_tweets(name, mode=modes, number=num)
    
    final_tweets = []
    if 'tweets' in total:
        for tweet in total['tweets']:  # Adjust the key to match actual data
            data = [tweet['link'], tweet['text'], tweet['date']]
            final_tweets.append(data)
        data = pd.DataFrame(final_tweets, columns=['link', 'text', 'date'])
        return data
    else:
        print("Key 'tweets' not found in response")
        return None

In [62]:
data = getTweets('mpox', 'term', 25)
print(data)

Testing instances: 100%|██████████| 16/16 [00:08<00:00,  1.80it/s]

21-Sep-24 16:33:18 - No instance specified, using random instance https://nitter.net


21-Sep-24 16:33:23 - https://nitter.net unreachable. Trying https://nitter.net
21-Sep-24 16:33:34 - https://nitter.net unreachable. Trying https://nitter.net
High number of retries detected. Testing all instances...


Testing instances: 100%|██████████| 16/16 [00:12<00:00,  1.24it/s]

New working instances: 
21-Sep-24 16:33:54 - All instances are unreachable. Check your request and try again.
Empty DataFrame
Columns: [link, text, date]
Index: []


In [23]:
df = pd.DataFrame(data[['link', 'date', 'text']])
df.to_json("tweets_filtered.json", orient='index')
df

,link,date,text
0,https://twitter.com/RaffaellaRegoli/status/183...,"Sep 19, 2024 · 9:27 AM UTC",#Covid 🚨++ARRIVANO I NUOVI VACCINI PER IL COVI...
1,https://twitter.com/TonixPharma/status/1836838...,"Sep 19, 2024 · 6:43 PM UTC",A major difference between the mpox outbreak s...
2,https://twitter.com/amerix/status/183666736473...,"Sep 19, 2024 · 7:23 AM UTC","PARENTS, Go find out if your children in scho..."
3,https://twitter.com/TabascoHOY/status/18369144...,"Sep 19, 2024 · 11:45 PM UTC",#México📍| ¿Vas a #Cancún? Q. Roo es el segundo...
4,https://twitter.com/CryptoPal4/status/18369163...,"Sep 19, 2024 · 11:52 PM UTC",Lol if you think that’s what the project is ab...
5,https://twitter.com/ChikatsuHayashi/status/183...,"Sep 19, 2024 · 1:29 PM UTC",★ FDAがシェディングの可能性を認める ★ FDAは、8月に承認したMpox(サル痘)ワ...
6,https://twitter.com/Monkeypoxtally/status/1836...,"Sep 19, 2024 · 6:14 PM UTC",The number of mpox cases in Africa has surged ...
7,https://twitter.com/ken24918164/status/1836915...,"Sep 19, 2024 · 11:50 PM UTC",速報 ! 米国 FDA 食品医薬品局は、エマージェントの MPOX「ワクチン」を使用して10...
8,https://twitter.com/heidijblake1/status/183691...,"Sep 19, 2024 · 11:50 PM UTC","Hi Kap, I was just watching your video about m..."
9,https://twitter.com/COVID19info101/status/1836...,"Sep 19, 2024 · 11:50 PM UTC",The number of mpox cases in Africa has jumped ...


In [24]:
!pip install firebase-admin


In [40]:
def upload_to_firebase(data):
    collection_ref = db.collection('tweets')

    records = data.to_dict(orient='records')
    
    for tweet in records:
        doc_ref = collection_ref.document() 
        doc_ref.set(tweet)

    print("Data uploaded to Firebase successfully!")

In [41]:
upload_to_firebase(df)

Data uploaded to Firebase successfully!
